In [ ]:
import segmentation_models as sm
import tensorflow as tf
import keras

import matplotlib.pyplot as plt

import numpy as np
import glob
import os
import cv2


# self-written scripts
import sys
sys.path.insert(0, 'Python_Scripts')

import data_preparation_cnn
import segmentation_modelling_cnn as smc


In [ ]:
# PARAMETERS
SIZE_X = 128
SIZE_Y = 512

EPOCHS = 20
LEARNING_RATE = 0.005
OPTIMIZER = tf.keras.optimizers.Adam(lr=LEARNING_RATE)

---
## Train New Models

With the following line of code, you can generate new models using varying parameters.

In [ ]:
models = smc.build_mask_generating_models(SIZE_X, SIZE_Y, 'dice', EPOCHS, OPTIMIZER)

---

## Using Our Pre-Trained Models

In the following we build use our pre-trained models (1 for each defect class) to demonstrate the outcome. Our fit used the following configuration:

- `Image Width` - 512 pixel
- `Image Height` - 128 pixel
- Model Architecture - a Unet with pre-trained `EfficientNetB5`
- `Loss Function` - Dice loss (custom function)
- `Metrics` - Dice coefficient (custom function)
- `Optimizer` - `adam` with a `learning_rate` of 0.005
- `Epochs` - 20

### Loading Saved Models

In [ ]:
models = smc.load_and_compile_dice_models_from_capstone()

### Predict Defect Masks

Using our pre-trained `models` we can generate mask images (1 per defect class) for each test image.

In [ ]:
predictions = smc.get_predictions_all_classes(models, SIZE_X, SIZE_Y)

### Visualize some Masks per Defect Class

Once the predicted masks are generated, we can have a look at some mask examples to get a feeling for the performance of our model.

In [ ]:
smc.visualize_predictions(predictions, 1, SIZE_X, SIZE_Y)

In [ ]:
smc.visualize_predictions(predictions, 2, SIZE_X, SIZE_Y)

In [ ]:
smc.visualize_predictions(predictions, 3, SIZE_X, SIZE_Y)

In [ ]:
smc.visualize_predictions(predictions, 4, SIZE_X, SIZE_Y)

Now let's have a look at the performance across defect classes side by side.

In [ ]:
smc.visualize_masks_across_defect_classes(predictions, SIZE_X, SIZE_Y)

---

### Calculating Classification Accuracy

As a measure of how well the masks that our models generate perform as a classifier for the recognized defect class we calculate our own accuracy rate. 

*Base assumption*: the mask that marks most pixels as defective reflects the defect class of the image.

This configuration rules out the identification of several different defects per image (multi-class).

In [ ]:
accuracies, weighted_acc = smc.calculate_accuracies_for_masks(models,SIZE_X, SIZE_Y)

Our model performs pretty well, given the imbalanced data set and heterogenous `ClassId` 3. With our model we are able to correctly localise and classify **76%** of images in our scenario.